In [1]:
!pip install -q opendatasets

import opendatasets as od
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torchvision.models import ResNet18_Weights
from torch.utils.data import DataLoader

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python3.9 -m pip install --upgrade pip


In [2]:
od.download('https://www.kaggle.com/datasets/puneet6060/intel-image-classification')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading intel-image-classification.zip to ./intel-image-classification


100%|██████████| 346M/346M [00:15<00:00, 24.1MB/s] 


In [2]:
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

inference_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [3]:
train_dataset = datasets.ImageFolder(root='/Users/luka/PyCharmProjects/simple-defect-detection/data/intel-image-classification/seg_train/seg_train', transform=train_transforms)
val_dataset = datasets.ImageFolder(root='/Users/luka/PyCharmProjects/simple-defect-detection/data/intel-image-classification/seg_test/seg_test', transform=inference_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
    
device

device(type='mps')

In [5]:
model = models.resnet18(weights=ResNet18_Weights.DEFAULT)

# Modify the final fully connected layer to match the number of classes in your new dataset
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 6)  # num_classes should be set to the number of your new categories

model = model.to(device)

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [7]:
num_epochs = 2

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Validation Loss: {val_loss/len(val_loader)}, Accuracy: {100 * correct / total}%")

print("Training complete")

KeyboardInterrupt: 

In [14]:
idx_to_class = {v: k for k, v in train_dataset.class_to_idx.items()}

def predict(img_path):
  image = Image.open(img_path)

  transformed_image = inference_transforms(image)
  transformed_image = transformed_image.unsqueeze(0)  # Add batch dimension
  transformed_image = transformed_image.to(device)

  model.eval()

  with torch.no_grad():
      outputs = model(transformed_image)
      _, predicted = torch.max(outputs, 1)
      predicted_class = idx_to_class[predicted.item()]

  print(f'Predicted class: {predicted_class}')

In [21]:
predict('/Users/luka/PyCharmProjects/simple-defect-detection/data/intel-image-classification/seg_pred/seg_pred/84.jpg')

Predicted class: mountain
